In [3]:
import cx_Oracle

user = "SCOTT"
pw = "TIGER"
dsn = "localhost:1521/orcl" # ip번호:host번호/SID

con = cx_Oracle.connect(user, pw, dsn, encoding="UTF-8")

print("Database version:", con.version)
# con.close() # 자원 반납

Database version: 19.3.0.0.0


In [4]:
# 1. 특정 레코드 하나 얻기. deptno=10
# 커서를 얻는 것(con.cursor()) -> DB영역에서 움직임
# 자동으로 커서 연결을 종료하기 위해 with문 사용

with con.cursor() as cur:   
    cur.execute("SELECT * FROM dept WHERE deptno =:x", x=10) # 바인딩 변수(Oracle에서만 사용 가능)
    res = cur.fetchone()
    print(res)
    
# 포맷팅 가능
# with con.cursor() as cur:
#     sql = "select * from dept where deptno ={xxx}".format(xxx=10)
#     cur.execute(sql)
#     res = cur.fetchone()
#     print(res)

(10, 'ACCOUNTING', 'NEW YORK')


In [7]:
# 3. 멀티 레코드 조회
with con.cursor() as cur:
    cur.execute("SELECT * FROM dept ORDER BY deptno")
    res = cur.fetchall()  # list로 반환
    print(res)
    for row in res:
        print(row)

[(10, 'ACCOUNTING', 'NEW YORK'), (20, 'RESEARCH', 'DALLAS'), (30, 'SALES', 'CHICAGO'), (40, 'OPERATIONS', 'BOSTON')]
(10, 'ACCOUNTING', 'NEW YORK')
(20, 'RESEARCH', 'DALLAS')
(30, 'SALES', 'CHICAGO')
(40, 'OPERATIONS', 'BOSTON')


In [8]:
# 4. 단일 저장
with con.cursor() as cur:
    cur.execute( "INSERT INTO dept (deptno, dname, loc) VALUES " \
                 " (:deptno, :dname, :loc)", deptno=99, dname='개발', loc="서울") # 바인딩 변수
    print("저장된 레코드갯수:", cur.rowcount) # 적용된 rowcount 불러온다
    con.commit()

저장된 레코드갯수: 1


In [10]:
# 결과 확인
with con.cursor() as cur:
    cur.execute("SELECT * FROM dept ORDER BY deptno")
    res = cur.fetchall()  # list로 반환
    for row in res:
        print(row)

(10, 'ACCOUNTING', 'NEW YORK')
(20, 'RESEARCH', 'DALLAS')
(30, 'SALES', 'CHICAGO')
(40, 'OPERATIONS', 'BOSTON')
(99, '개발', '서울')


In [11]:
# 5. 멀티 저장
with con.cursor() as cur:
    rows = [(1, "개발","서울"), (2, "개발","서울")]
    
    # MariaDB는 :1, :2, :3이 아니라 ?, ?, ?
    cur.executemany("INSERT INTO dept (deptno, dname, loc) VALUES  (:1, :2, :3)", rows)
    print("저장된 레코드갯수:", cur.rowcount)
    con.commit()

저장된 레코드갯수: 2


In [12]:
# 결과 확인
with con.cursor() as cur:
    cur.execute("SELECT * FROM dept ORDER BY deptno")
    res = cur.fetchall()  # list로 반환
    for row in res:
        print(row)

(1, '개발', '서울')
(2, '개발', '서울')
(10, 'ACCOUNTING', 'NEW YORK')
(20, 'RESEARCH', 'DALLAS')
(30, 'SALES', 'CHICAGO')
(40, 'OPERATIONS', 'BOSTON')
(99, '개발', '서울')


In [13]:
# 6. 수정
with con.cursor() as cur:
    cur.execute( "UPDATE dept SET dname = :x, loc= :y "
                 " WHERE deptno = :z", x="개발부", y="서울시", z=1 )
    print("수정된 레코드갯수:", cur.rowcount)
    con.commit()

수정된 레코드갯수: 1


In [14]:
# 결과 확인
with con.cursor() as cur:
    cur.execute("SELECT * FROM dept ORDER BY deptno")
    res = cur.fetchall()  # list로 반환
    for row in res:
        print(row)

(1, '개발부', '서울시')
(2, '개발', '서울')
(10, 'ACCOUNTING', 'NEW YORK')
(20, 'RESEARCH', 'DALLAS')
(30, 'SALES', 'CHICAGO')
(40, 'OPERATIONS', 'BOSTON')
(99, '개발', '서울')


In [15]:
# 7. 삭제
with con.cursor() as cur:
    cur.execute( "DELETE FROM dept WHERE deptno = :z",  z=2 )
    print("삭제된 레코드갯수:", cur.rowcount)
    con.commit() 

삭제된 레코드갯수: 1


In [16]:
# 결과 확인
with con.cursor() as cur:
    cur.execute("SELECT * FROM dept ORDER BY deptno")
    res = cur.fetchall()  # list로 반환
    for row in res:
        print(row)

(1, '개발부', '서울시')
(10, 'ACCOUNTING', 'NEW YORK')
(20, 'RESEARCH', 'DALLAS')
(30, 'SALES', 'CHICAGO')
(40, 'OPERATIONS', 'BOSTON')
(99, '개발', '서울')


In [17]:
# 원래대로 복귀
with con.cursor() as cur:
    cur.execute( "DELETE FROM dept WHERE deptno = :z",  z=1 )
    print("삭제된 레코드갯수:", cur.rowcount)
    cur.execute( "DELETE FROM dept WHERE deptno = :z",  z=99 )
    print("삭제된 레코드갯수:", cur.rowcount)
    con.commit() 

삭제된 레코드갯수: 1
삭제된 레코드갯수: 1


In [18]:
# 결과 확인
with con.cursor() as cur:
    cur.execute("SELECT * FROM dept ORDER BY deptno")
    res = cur.fetchall()  # list로 반환
    for row in res:
        print(row)

(10, 'ACCOUNTING', 'NEW YORK')
(20, 'RESEARCH', 'DALLAS')
(30, 'SALES', 'CHICAGO')
(40, 'OPERATIONS', 'BOSTON')


In [19]:
con.close() # 자원 반납